# Evaluation of Drone Trajectory Planning with the Various Initial Guesses

This analysis compiles results from evaluating drone trajectory planning solutions for a range of operational scenarios using multi-segment pseudospectral optimal control techniques. The computational work was distributed across MetaCentrum clusters, therefore the focus is not on convergence time but rather the quality of achieved solutions. The key metrics we will assess across scenarios are number of iterations, number of collocation points, cost function values, and constraint violations representing trajectory efficiency. The goal is to compare solution consistency across the complexities introduced through world configurations and initial solver guesses.

The raw output data is aggregated into individual CSVs for each planning run specification. Python Pandas ingests these files into separate dataframes then concatenates them into a unified table for analysis. Appropriate data normalization and preprocessing is applied to ensure consistency prior to statistical evaluation. The following key steps will be taken:

1) Importing and combining cluster output CSVs
2) Cleansing composite dataframe
3) Analysis of performance trends across planning scenarios
4) Recommendations for formulating future problems to improve consistency

In [12]:
import numpy as np
import pandas as pd
from IPython.display import display
import csv
import os
import re
import cloudpickle as cp

## Access the Dataset & Create Pandas Dataframe

In [13]:
# set path to results
current_folder = os.getcwd()
results_dir = "results"
data_dir = "data"
eval_dir = "eval"
init_dir = "inits"
traj_dir = "trajs"
table_dir = "tables"

full_path_eval = os.path.join(current_folder, results_dir, data_dir, eval_dir)
csv_files_eval = os.listdir(full_path_eval)
full_path_table = os.path.join(current_folder, results_dir, table_dir)
full_path_traj = os.path.join(current_folder, results_dir, data_dir, traj_dir)
pickle_files_traj = os.listdir(full_path_traj)

In [14]:
# describe patterns
worlds = ["simple", "simple2", "orchard", "columns", "random_spheres",
          "forest", "random_columns", "walls"]
worlds_results = worlds #["simple", "simple2", "columns", "random_spheres", "random_columns"]
init_levels = ["none", "position", "speed", "orientation", "angular_rate"]
world_pattern = r"|".join(re.escape(w) for w in worlds)
init_pattern = r"|".join(re.escape(w) for w in init_levels)
pattern = re.compile(
    r"ps_log_(?P<name>" + world_pattern + r")_"       # Match world name group 
    r"(?P<bound>bound(on|off))_"
    r"(?P<const>const(on|off))_"
    r"(?P<init_level>""" + init_pattern + r")_"
    r"(?P<ctrl>ctrl(on|off))_"
    r"(?P<init_domain>(isingle|imulty))_"
    r"(?P<seg>(single|multi))_"
    r"(?P<approx>\w+).csv"
)

In [15]:
data = []
for f in csv_files_eval:
    # go through each file with evaluation of trajectory and planning procedure
    match = pattern.search(f)
    if match:
        # patterns found
        groups = match.groupdict()
        name = groups["name"]
        init_level = groups["init_level"]
        bound = groups["bound"]
        const = groups["const"] 
        ctrl = groups["ctrl"]
        init_domain = groups["init_domain"]
        seg = groups["seg"]
        approx = groups["approx"]
        bound_and_const = "Forced Bound. and Const."
        
        # read CSV file and process data + labels
        reader = pd.read_csv(os.path.join(full_path_eval,f))
        if len(data) == 0:
            data_head = ["World Name", "Init. Level", bound_and_const, "Forced Constraints", "Init. Control", "Init. Domain", "Solution Domain", "Approximation", "Filename"] + list(reader)
        data.append([name, init_level, bound, const, ctrl, init_domain, seg, approx, f]+list(reader.iloc[0,:]))

df_eval = pd.DataFrame(data, columns=data_head)

## Clean & Convert Data

In [16]:
# recalculate errors with higher precision
new_n_simpson = 10 # number of points for Simpson's rule (infuences accuracy of numerical integral)

pattern_solution = re.compile(
    r"ps_sol_(?P<name>" + world_pattern + r")_"       # Match world name group 
    r"(?P<bound>bound(on|off))_"
    r"(?P<const>const(on|off))_"
    r"(?P<init_level>""" + init_pattern + r")_"
    r"(?P<ctrl>ctrl(on|off))_"
    r"(?P<init_domain>(isingle|imulty))_"
    r"(?P<seg>(single|multi))_"
    r"(?P<approx>\w+).pkl"
)

def recalc_errors(f):
    match = pattern_solution.search(f)
    if match:
        item_file = f
        item_file = item_file.replace('ps_sol_', 'ps_log_')
        matching_indices = df_eval[df_eval['Filename'].str.contains(item_file)].index
        with open(os.path.join(full_path_traj,f), mode='rb') as file:
            groups = match.groupdict()
            sol = cp.load(file)
            sol.evaluateSolution(new_n_simpson)
            constraints_error = sol.get_constraints_error() #  "Total Violation", "State Violation", "Control Violation", "Obstacles Violation"
            abs_error = np.max(np.concatenate(sol.max_error))
            df_eval.loc[matching_indices, 'Absolute Error'] = abs_error
            df_eval.loc[matching_indices, 'Total Violation'] = constraints_error[0]
            df_eval.loc[matching_indices, 'State Violation'] = constraints_error[1]
            df_eval.loc[matching_indices, 'Control Violation'] = constraints_error[2]
            df_eval.loc[matching_indices, 'Obstacles Violation'] = constraints_error[3]

In [17]:
if False:
    import multiprocessing
    # go through each file with evaluation of trajectory and planning procedure
    pool = multiprocessing.Pool(processes=len(pickle_files_traj))  
    pool.map(recalc_errors, pickle_files_traj)

In [18]:
init_levels_print = ["none", "position", "velocity", "orientation", "angular rate"]
# rename columns
df_eval.rename(columns={'Objective': 'Optimality Criterion'}, inplace=True)
# rename values
df_eval[bound_and_const] = df_eval[bound_and_const].replace({'boundon': 'yes', 'boundoff': 'no'})
df_eval['Init. Control'] = df_eval['Init. Control'].replace({'ctrlon': 'yes', 'ctrloff': 'no'})
df_eval['Init. Level'] = df_eval['Init. Level'].replace({'angular_rate': 'angular rate', 
                                                        'speed':'velocity'})
df_eval['Init. Domain'] = df_eval['Init. Domain'].replace({'isingle': 'single', 'imulty': 'multi'})
# sort according to initialization level
df_eval['Init. Level'] = pd.Categorical(df_eval['Init. Level'], categories=init_levels_print, ordered=True)
df_eval = df_eval.sort_values(by=['Init. Level', 'Iteration No.', 'Total Time'])
df_eval.describe()

,Iteration No.,Optimality Criterion,Absolute Error,Relative Error,Total Violation,State Violation,Control Violation,Obstacles Violation,Total Time,Iter. Time
count,112.000000,112.000000,112.000000,112.000000,1.120000e+02,112.000000,112.000000,112.000000,112.000000,112.000000
mean,4.366071,25.955521,0.005861,3.275916,1.577020e-02,0.001838,0.000408,0.013524,621.677179,183.641138
std,2.983424,9.564064,0.002561,1.372988,1.995572e-02,0.002875,0.000872,0.017936,1366.028092,191.347218
min,1.000000,12.741786,0.001168,1.776502,3.196288e-11,0.000000,0.000000,0.000000,21.264973,10.823604
25%,2.000000,17.537854,0.003904,2.691247,4.268949e-03,0.000012,0.000000,0.003305,107.657845,37.506036
50%,4.000000,25.661624,0.005868,3.071238,1.216281e-02,0.000853,0.000053,0.009382,207.593249,133.798291
75%,6.000000,30.451018,0.008008,3.509293,2.097855e-02,0.002494,0.000300,0.016549,503.689017,259.493369
max,16.000000,56.241607,0.009998,12.316539,1.399558e-01,0.017575,0.006039,0.122255,8071.609508,901.791524


## Examples of filtering tables

In [19]:
# df_eval_world = df_eval.loc[df_eval['World Name'] == worlds_results[1]].sort_values(by=['Total Time'])[['World Name', 'Init. Level', 'Init. Control', bound_and_const, 'Init. Domain', 'Solution Domain', 
#                                                                                                      'Segmentation', 'Iteration No.', 'Absolute Error', 'Optimality Criterion', 
#                                                                                                      'Total Violation', 'Obstacles Violation', 'Total Time']]
# print(df_eval_world.count())
# df_eval_world

# df_eval_world.describe()

In [20]:
# # Convert the 'Init. Level' column to a categorical type with the specified order
# df_eval_world['Init. Level'] = pd.Categorical(df_eval_world['Init. Level'], categories=init_levels_print, ordered=True)

# # Now sort the DataFrame by the 'Init. Level' column
# df_sorted = df_eval_world.sort_values('Init. Level')
# df_sorted

## Color the Cells

In [21]:
color_palette = {
    'color1': '#529DCB',
    'color2': '#ECA063',
    'color3': '#71BF50',
    'color4': '#F3CC4F',
    'color5': '#D46934',
    'color6': '#A1D8B6',
    'color7': '#D2C48E',
    'color8': '#F45F40',
    'color9': '#F9AE8D',
    'color10': '#80B9CE'
}

def get_styled_df(df, columns_to_style = None, print_color = False):
    """Get styled dataframe according to extremas and quantiles"""
    
    # Define columns to style
    if columns_to_style is None:
        columns_to_style = ['Iteration No.', 'Optimality Criterion', 'Absolute Error', 'Total Violation', 'Obstacles Violation', 'Total Time']

    # Compute quantiles and extrema
    quantiles = df[columns_to_style].quantile([0.10, 0.90])
    min_vals = df[columns_to_style].min()
    max_vals = df[columns_to_style].max()

    # Define a function to apply color based on quantiles and extrema
    if print_color:
        def color_cells(column):
            return [
                'background-color: \\cellcolor{color3};' if val == min_vals[column.name] else
                'background-color: \\cellcolor{color8};' if val == max_vals[column.name] else
                'background-color: \\cellcolor{color10};' if val <= quantiles.loc[0.10, column.name] else
                'background-color: \\cellcolor{color9};' if val >= quantiles.loc[0.90, column.name] else
                ''
                for val in column
            ]
    else:
        def color_cells(column):
            return [
                f'background-color: {color_palette["color3"]};' if val == min_vals[column.name] else
                f'background-color: {color_palette["color8"]};' if val == max_vals[column.name] else
                f'background-color: {color_palette["color10"]};' if val <= quantiles.loc[0.10, column.name] else
                f'background-color: {color_palette["color9"]};' if val >= quantiles.loc[0.90, column.name] else
                ''
                for val in column
            ]

    # Apply the styling
    styled_df = df.style.apply(color_cells, subset=columns_to_style)
    return styled_df

def create_styled_dfs(df, worlds, filter_rule, columns_to_style = None, print_color=False):
    """
    Create a list of styled dataframes based on a given filter rule.

    :param df: The original DataFrame.
    :param worlds: A list of 'worlds' to filter by.
    :param filter_rule: A function that takes a DataFrame and returns a filtered DataFrame.
    :param print_color: Flag to control color printing in styling.
    :return: A tuple of two lists - styled dataframes and their corresponding world names.
    """
    dfs_styled = []
    world_names = []

    for world in worlds:
        df_filtered = filter_rule(df[df['World Name'] == world].drop(columns=['World Name']))
        df_filtered = df_filtered.reset_index(drop=True)
        # print(world)
        styled_df = get_styled_df(df_filtered, columns_to_style = columns_to_style, print_color=print_color)
        dfs_styled.append(styled_df)
        world_names.append(world)
        # display(styled_df)

    return dfs_styled, world_names

def rule_single_domain(df):
    # Define your custom filter rule here, for example:
    return df[df['Init. Domain'] == 'single'].drop(columns=['Init. Domain'])

def rule_multi_domain(df):
    # Define your custom filter rule here, for example:
    return df[df['Init. Domain'] == 'multi'].drop(columns=['Init. Domain'])

In [22]:
# clean cells and set printing to latex
df_eval_clean = df_eval.drop(columns=['Filename', 'Approximation', 'Iter. Time','Relative Error', 'Forced Constraints','Segmentation','State Violation','Control Violation'])
# new init level with control
if 'angular rate with control' not in df_eval_clean['Init. Level'].cat.categories:
    df_eval_clean['Init. Level'] = df_eval_clean['Init. Level'].cat.add_categories(['ang. rate ctrl'])
df_eval_clean.loc[(df_eval_clean['Init. Control'] == 'yes') & (df_eval_clean['Init. Level'] == 'angular rate'), 'Init. Level'] = 'ang. rate ctrl'
df_eval_clean = df_eval_clean.drop(columns=['Init. Control'])
# rename column
df_eval_clean = df_eval_clean.rename(columns={'Forced Bound. and Const.': 'Constraints'})
# sort again
init_levels_print_new = ["none", "position", "velocity", "orientation", "angular rate", "ang. rate ctrl"]
df_eval_clean['Init. Level'] = pd.Categorical(df_eval_clean['Init. Level'], categories=init_levels_print_new, ordered=True)
df_eval_clean = df_eval_clean.sort_values(by=['Init. Level', 'Iteration No.', 'Total Time'])
df_eval_clean = df_eval_clean.loc[~((df_eval_clean['Init. Level']=='none') & (df_eval_clean['Constraints'] == 'yes'))]
df_eval_clean = df_eval_clean.loc[~(df_eval_clean['Iteration No.']>9)]

In [23]:
# get lists of styled dataframes
dfs_single, __ = create_styled_dfs(df_eval_clean, worlds_results, rule_single_domain, None, False)
dfs_multi, name_temp = create_styled_dfs(df_eval_clean, worlds_results, rule_multi_domain, None, False)

In [24]:
# show dataframes
for df, df2, name in zip(dfs_single, dfs_multi, name_temp):
    print(name, 'single')
    display(df)
    print(name, 'multi')
    display(df2)

simple single


,Init. Level,Constraints,Solution Domain,Iteration No.,Optimality Criterion,Absolute Error,Total Violation,Obstacles Violation,Total Time
0,position,yes,single,6,44.226072,0.004162,0.005474,0.003686,124.167714
1,position,no,single,6,44.226072,0.004162,0.005474,0.003686,129.128501
2,velocity,yes,multi,6,42.669896,0.008012,0.024740,0.017904,696.541319
3,velocity,yes,single,8,40.739350,0.006631,0.004269,0.002247,156.331508
4,orientation,yes,multi,3,40.955713,0.001559,0.000000,0.000000,296.977610
5,orientation,yes,single,7,41.189126,0.009998,0.006357,0.002988,115.080537
6,angular rate,yes,multi,6,42.669896,0.008012,0.024740,0.017904,666.515347
7,angular rate,yes,single,8,40.739350,0.006631,0.004269,0.002247,205.538884
8,ang. rate ctrl,yes,multi,6,39.702934,0.007035,0.021439,0.014475,691.338625
9,ang. rate ctrl,yes,single,8,41.283958,0.007576,0.004540,0.002190,155.842089


simple multi


,Init. Level,Constraints,Solution Domain,Iteration No.,Optimality Criterion,Absolute Error,Total Violation,Obstacles Violation,Total Time
0,position,yes,multi,1,37.782656,0.005153,0.139956,0.122255,25.166406
1,position,no,multi,1,37.782656,0.005153,0.139956,0.122255,31.599647
2,velocity,yes,multi,4,40.720256,0.007293,0.020979,0.014270,504.418521
3,angular rate,yes,multi,4,40.720256,0.007293,0.020979,0.014270,522.783421
4,ang. rate ctrl,yes,multi,3,44.106980,0.007367,0.024443,0.020545,351.949287


simple2 single


...

,Init. Level,Constraints,Solution Domain,Iteration No.,Optimality Criterion,Absolute Error,Total Violation,Obstacles Violation,Total Time
0,none,no,multi,4,22.641502,0.007442,0.000210,0.000129,1576.182860
1,position,no,multi,1,14.922155,0.002957,0.039037,0.038940,31.196077
2,position,yes,multi,1,14.922155,0.002957,0.039037,0.038940,37.944117
3,velocity,yes,multi,2,24.455775,0.009371,0.014934,0.009258,345.937760
4,orientation,yes,multi,3,16.176334,0.006394,0.000881,0.000796,311.260024
5,angular rate,yes,multi,2,24.455775,0.009371,0.014934,0.009258,338.723431
6,ang. rate ctrl,yes,multi,3,17.582212,0.004661,0.015342,0.010081,346.072877


walls single


,Init. Level,Constraints,Solution Domain,Iteration No.,Optimality Criterion,Absolute Error,Total Violation,Obstacles Violation,Total Time
0,none,no,multi,2,16.772768,0.003287,0.015125,0.015122,81.959162
1,none,no,single,6,15.332157,0.006595,0.012163,0.012163,98.843291
2,position,yes,multi,5,16.116317,0.004731,0.045643,0.042097,351.791955
3,position,no,multi,5,16.116317,0.004731,0.045643,0.042097,542.001790
4,position,yes,single,6,15.375581,0.005868,0.012945,0.012881,107.821199
5,position,no,single,6,15.375581,0.005868,0.012945,0.012881,118.698669
6,velocity,no,single,2,24.232950,0.006294,0.016759,0.016260,21.264973
7,velocity,no,multi,2,19.906748,0.009666,0.003582,0.002043,123.584977
8,velocity,yes,single,3,18.047949,0.002856,0.017174,0.014717,28.371757
9,velocity,yes,multi,4,22.334270,0.002566,0.004193,0.004038,808.549309


walls multi


,Init. Level,Constraints,Solution Domain,Iteration No.,Optimality Criterion,Absolute Error,Total Violation,Obstacles Violation,Total Time
0,none,no,multi,1,13.171676,0.001170,0.043053,0.041456,77.247996
1,position,yes,multi,1,14.803698,0.006684,0.009506,0.009506,187.306975
2,position,no,multi,1,14.803698,0.006684,0.009506,0.009506,387.325201
3,velocity,yes,multi,3,21.500553,0.003016,0.002648,0.002648,321.876970
4,velocity,no,multi,3,13.808265,0.004153,0.012164,0.011938,1663.786742
5,orientation,yes,multi,3,17.132038,0.007614,0.008605,0.006434,592.400377
6,orientation,no,multi,3,12.741786,0.009413,0.017126,0.015408,657.856814
7,angular rate,yes,multi,3,21.500553,0.003016,0.002648,0.002648,525.200876
8,angular rate,no,multi,3,13.808265,0.004153,0.012164,0.011938,1133.862811
9,ang. rate ctrl,yes,multi,2,56.241607,0.007947,0.005215,0.004524,420.237794


In [25]:
def print_latex(df, world_name, init_domain, full_path_table):
    os.makedirs(full_path_table, exist_ok=True)
    buf = os.path.join(full_path_table,"table_traj_plan_eval_"+world_name+"_"+init_domain+".tex")
    if world_name == 'simple2':
        caption = "Evaluation of UAV trajectories found by PSM and PSEM for 2 obstacles with "+init_domain+"-segment initialization."
        label = "tab:traj-plan-eval-"+world_name+"-"+init_domain
    elif world_name == 'simple':
        caption = "Evaluation of UAV trajectories found by PSM and PSEM for 1 obstacle with "+init_domain+"-segment initialization."
        label = "tab:traj-plan-eval-"+world_name+"-"+init_domain
    else:
        caption = "Evaluation of UAV trajectories found by PSM and PSEM for "+world_name.replace("_", " ")+" with "+init_domain+"-segment initialization."
    label = "tab:traj-plan-eval-"+world_name+"-"+init_domain
    # column_format = 'p{25mm}' +''.join(['p{15mm}' for _ in range(len(df.columns)-1)])
    column_format = 'p{21mm}p{9mm}p{12mm}p{6mm}p{14mm}p{14mm}p{14mm}p{14mm}p{14mm}'
    df.format({
            'Optimality Criterion':'{:,.2e}',
            'Absolute Error':'{:,.2e}',
            'Sum Viol.':'{:,.2e}',
            'State Violation':'{:,.2e}',
            'Control Violation':'{:,.2e}',
            'Obstacle Viol.':'{:,.2e}',
            'Total Time': lambda x: f'{x:.2f}s'
        })
    df.format_index(axis=1, formatter="{}".format).hide(axis=0)
    latex_code = df.to_latex(caption = caption, 
                label = label, column_format = column_format, hrules=True)#, index=False)
    corrected_latex_code = latex_code.replace('\\background-color', '')
    # Insert \small command
    begin_table_index = corrected_latex_code.find('\\begin{table}')
    if begin_table_index != -1:
        # Insert \small right after \begin{table}
        corrected_latex_code = corrected_latex_code[:begin_table_index + 13] + "\n\\small\n" + corrected_latex_code[begin_table_index + 13:]

    with open(buf, 'w') as f:
        f.write(corrected_latex_code)

In [26]:
# shorter names
df_eval_clean_print = df_eval_clean.rename(columns={'Constraints': 'Constr.','Solution Domain':'Method','Iteration No.':'Iter.','Total Violation':'Sum Viol.','Obstacles Violation':'Obstacle Viol.'})
columns_to_style = ['Iter.', 'Optimality Criterion', 'Absolute Error', 'Sum Viol.', 'Obstacle Viol.', 'Total Time']
# get lists of styled dataframes
dfs_single, __ = create_styled_dfs(df_eval_clean_print, worlds_results, rule_single_domain, columns_to_style, True)
dfs_multi, name_temp = create_styled_dfs(df_eval_clean_print, worlds_results, rule_multi_domain, columns_to_style, True)

for df, world_name in zip(dfs_single, worlds_results):
    print_latex(df, world_name, 'single', full_path_table)
for df, world_name in zip(dfs_multi, worlds_results):
    print_latex(df, world_name, 'multi', full_path_table)